<a href="https://colab.research.google.com/github/taddbackus/capstone/blob/main/Testing_the_clean_up_alt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install keras

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [6]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib.pyplot as plt

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [8]:

all_mines = pd.read_csv('/content/drive/MyDrive/Mines_Data_Clean/all_mines_no_filter_with_precip.csv')
all_mines = pd.DataFrame(all_mines)

<ipython-input-8-0d4bec27f980>:1: DtypeWarning: Columns (6,16,18,19,20,21,35,36,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  all_mines = pd.read_csv('/content/drive/MyDrive/Mines_Data_Clean/all_mines_no_filter_with_precip.csv')


In [9]:
columns_to_drop = ['Unnamed: 0']
all_mines = all_mines.drop(columns = columns_to_drop)


all_mines.columns

Index(['Company', 'start_date', 'Observations', 'orig_perm_id', 'mine_id',
       'State', 'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
       'shrub_and_scrub', 'built', 'bare', 'snow_and_ice', 'company',
       'coalmine_op_status', 'mine_name', 'permit_id', 'national_id',
       'coal_bed_names', 'inspectable_unit_status', 'post_smcra',
       'reported_area', 'permit_application_type', 'area_mine', 'contour',
       'mountaintop', 'steep_slope', 'highwall', 'auger', 'contact',
       'Number_company_to_permit_id', 'edit_month', 'edit_year',
       'permit_weekday', 'permit_approval_month', 'permit_approval_year',
       'permit_application_year', 'WaterQuality', 'Max Temperature',
       'Precipitation'],
      dtype='object')

In [10]:
all_mines = all_mines.sort_values(by=['permit_id', 'start_date']).reset_index()

In [11]:
#all_mines = pd.read_csv('/Users\\austi\\Documents\\Capstone\\Working\\For real\\Capstone Data\\all_mines.csv')

all_mines.head(15)

,index,Company,start_date,Observations,orig_perm_id,mine_id,State,water,trees,grass,...,Number_company_to_permit_id,edit_month,edit_year,permit_weekday,permit_approval_month,permit_approval_year,permit_application_year,WaterQuality,Max Temperature,Precipitation
0,13464,TENNESSEE KENTUCKY COAL CO,2019-07-01,8.0,0070010,00000000000000001e70,ky,0.029045,0.747375,0.031408,...,0.0,11.0,1999.0,Saturday,12.0,1899,NaT,-0.266105,300.600006,28.100000
1,18385,TENNESSEE KENTUCKY COAL CO,2021-07-01,4.0,0070010,00000000000000001e70,ky,0.032193,0.516882,0.051611,...,0.0,11.0,1999,Saturday,12.0,1899,NaT,-0.588503,299.100006,56.200001
2,3821,TENNESSEE KENTUCKY COAL CO,2023-07-01,4.0,0070010,00000000000000001e70,ky,0.021954,0.670695,0.064824,...,0.0,11.0,1999.0,Saturday,12.0,1899,NaT,0.056293,302.100006,0.000000
3,14135,WATERS COAL CO,2019-07-01,4.0,0070015,00000000000000001e71,ky,0.030826,0.733858,0.035286,...,0.0,11.0,1999.0,Saturday,12.0,1899,NaT,-0.221152,301.799988,0.000000
4,19064,WATERS COAL CO,2021-07-01,4.0,0070015,00000000000000001e71,ky,0.031337,0.740100,0.033135,...,0.0,11.0,1999,Saturday,12.0,1899,NaT,-0.705038,298.399994,53.000000
5,4504,WATERS COAL CO,2023-07-01,4.0,0070015,00000000000000001e71,ky,0.029417,0.744496,0.034142,...,0.0,11.0,1999.0,Saturday,12.0,1899,NaT,0.051123,301.600006,0.000000
6,14136,MAGNUM EQUIPMENT CO,2019-07-01,6.0,0070017,00000000000000001e72,ky,0.033030,0.731213,0.036031,...,0.0,11.0,1999.0,Saturday,12.0,1899,NaT,-0.632698,303.500000,0.000000
7,19065,MAGNUM EQUIPMENT CO,2021-07-01,12.0,0070017,00000000000000001e72,ky,0.033021,0.730987,0.034570,...,0.0,11.0,1999,Saturday,12.0,1899,NaT,-0.685335,300.600006,67.500000
8,4505,MAGNUM EQUIPMENT CO,2023-07-01,6.0,0070017,00000000000000001e72,ky,0.032437,0.736611,0.035406,...,0.0,11.0,1999.0,Saturday,12.0,1899,NaT,0.031698,303.299988,0.000000
9,25337,T C BELL INC,2017-07-01,15.0,0070019,00000000000000001e73,ky,0.026459,0.753329,0.031344,...,0.0,11.0,1999,Saturday,12.0,1899,NaT,-0.531245,302.024994,16.725000


In [12]:
all_mines['start_date'].unique()

array(['2019-07-01', '2021-07-01', '2023-07-01', '2017-07-01'],
      dtype=object)

In [13]:
all_mines.dtypes

index                            int64
Company                         object
start_date                      object
Observations                   float64
orig_perm_id                    object
mine_id                         object
State                           object
water                          float64
trees                          float64
grass                          float64
flooded_vegetation             float64
crops                          float64
shrub_and_scrub                float64
built                          float64
bare                           float64
snow_and_ice                   float64
company                         object
coalmine_op_status             float64
mine_name                       object
permit_id                       object
national_id                     object
coal_bed_names                  object
inspectable_unit_status        float64
post_smcra                     float64
reported_area                  float64
permit_application_type  

In [14]:
for column_name, dtype in all_mines.dtypes.items():
    if dtype == 'object':
        column_values = all_mines[column_name]

        num_unique_values = column_values.nunique()

        print(f'Column: {column_name}')
        print(f'Number of Unique Values: {num_unique_values}\n')
        value_counts = column_values.value_counts()
        print(value_counts)
        print('-------------------------------------------\n')


Column: Company
Number of Unique Values: 3065

REVELATION ENERGY LLC                525
MARTIN COUNTY COAL CORPORATION       389
LCC KENTUCKY LLC                     362
NALLY & HAMILTON ENTERPRISES INC     320
ICG HAZARD LLC                       318
                                    ... 
LOGAN FORK COAL COMPANY                1
J & N PROCESSING COMPANY, L.L.C.       1
PERFORMANCE COAL COMPANY, LLC          1
CRADDOCK & SON COAL COMPANY, INC.      1
SPARKY MINING COMPANY                  1
Name: Company, Length: 3065, dtype: int64
-------------------------------------------

Column: start_date
Number of Unique Values: 4

2019-07-01    9995
2021-07-01    9702
2017-07-01    9597
2023-07-01    9368
Name: start_date, dtype: int64
-------------------------------------------

Column: orig_perm_id
Number of Unique Values: 8860

8970346    24
0130182    24
0070128    24
0268001    24
8980437    24
           ..
U003600     1
0985412     1
S303187     1
S302791     1
R067100     1
Name: orig

In [15]:
columns_to_encode = ['Company', 'start_date']

all_mines_encoded = pd.get_dummies(all_mines, columns=columns_to_encode)
object_columns = all_mines_encoded.select_dtypes(include=['object'])
all_mines_encoded = all_mines_encoded.drop(columns=object_columns.columns)


In [16]:
all_mines_encoded.columns

Index(['index', 'Observations', 'water', 'trees', 'grass',
       'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built', 'bare',
       ...
       'Company_ZEMA MINING COMPANY', 'Company_ZEPHYER COAL COMPANY INC',
       'Company_ZEPHYR MINING CO', 'Company_ZINN COAL CORPORATION',
       'Company_ZODIAC MINING, INC.', 'Company_ZY COAL COMPANY',
       'start_date_2017-07-01', 'start_date_2019-07-01',
       'start_date_2021-07-01', 'start_date_2023-07-01'],
      dtype='object', length=3098)

In [17]:
for column_name in all_mines_encoded.columns:
    if all_mines_encoded[column_name].dtype == 'int64':
        all_mines_encoded[column_name] = all_mines_encoded[column_name].astype('float32')

all_mines_encoded

,index,Observations,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,built,bare,...,Company_ZEMA MINING COMPANY,Company_ZEPHYER COAL COMPANY INC,Company_ZEPHYR MINING CO,Company_ZINN COAL CORPORATION,"Company_ZODIAC MINING, INC.",Company_ZY COAL COMPANY,start_date_2017-07-01,start_date_2019-07-01,start_date_2021-07-01,start_date_2023-07-01
0,13464.0,8.0,0.029045,0.747375,0.031408,0.025280,0.029814,0.036664,0.031500,0.036222,...,0,0,0,0,0,0,0,1,0,0
1,18385.0,4.0,0.032193,0.516882,0.051611,0.027663,0.111608,0.079988,0.036708,0.036163,...,0,0,0,0,0,0,0,0,1,0
2,3821.0,4.0,0.021954,0.670695,0.064824,0.024527,0.044877,0.069876,0.029673,0.029886,...,0,0,0,0,0,0,0,0,0,1
3,14135.0,4.0,0.030826,0.733858,0.035286,0.027421,0.031867,0.039150,0.032288,0.036838,...,0,0,0,0,0,0,0,1,0,0
4,19064.0,4.0,0.031337,0.740100,0.033135,0.028886,0.030761,0.035582,0.031605,0.035315,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38657,27248.0,2.0,0.032422,0.547989,0.056670,0.044381,0.066788,0.074948,0.055281,0.041408,...,0,0,0,0,0,0,0,0,0,1
38658,27305.0,3.0,0.027210,0.740494,0.035314,0.025197,0.033700,0.038406,0.032508,0.033599,...,0,0,0,0,0,0,0,0,0,1
38659,27368.0,6.0,0.027466,0.693140,0.046976,0.027153,0.042630,0.053391,0.033973,0.030361,...,0,0,0,0,0,0,0,0,0,1
38660,27618.0,3.0,0.032560,0.196347,0.048224,0.029943,0.048412,0.037446,0.360315,0.045383,...,0,0,0,0,0,0,0,0,0,1


In [18]:
all_mines_encoded = all_mines_encoded.replace({True: 1, False: 0})

all_mines_encoded

,index,Observations,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,built,bare,...,Company_ZEMA MINING COMPANY,Company_ZEPHYER COAL COMPANY INC,Company_ZEPHYR MINING CO,Company_ZINN COAL CORPORATION,"Company_ZODIAC MINING, INC.",Company_ZY COAL COMPANY,start_date_2017-07-01,start_date_2019-07-01,start_date_2021-07-01,start_date_2023-07-01
0,13464.0,8.0,0.029045,0.747375,0.031408,0.025280,0.029814,0.036664,0.031500,0.036222,...,0,0,0,0,0,0,0,1,0,0
1,18385.0,4.0,0.032193,0.516882,0.051611,0.027663,0.111608,0.079988,0.036708,0.036163,...,0,0,0,0,0,0,0,0,1,0
2,3821.0,4.0,0.021954,0.670695,0.064824,0.024527,0.044877,0.069876,0.029673,0.029886,...,0,0,0,0,0,0,0,0,0,1
3,14135.0,4.0,0.030826,0.733858,0.035286,0.027421,0.031867,0.039150,0.032288,0.036838,...,0,0,0,0,0,0,0,1,0,0
4,19064.0,4.0,0.031337,0.740100,0.033135,0.028886,0.030761,0.035582,0.031605,0.035315,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38657,27248.0,2.0,0.032422,0.547989,0.056670,0.044381,0.066788,0.074948,0.055281,0.041408,...,0,0,0,0,0,0,0,0,0,1
38658,27305.0,3.0,0.027210,0.740494,0.035314,0.025197,0.033700,0.038406,0.032508,0.033599,...,0,0,0,0,0,0,0,0,0,1
38659,27368.0,6.0,0.027466,0.693140,0.046976,0.027153,0.042630,0.053391,0.033973,0.030361,...,0,0,0,0,0,0,0,0,0,1
38660,27618.0,3.0,0.032560,0.196347,0.048224,0.029943,0.048412,0.037446,0.360315,0.045383,...,0,0,0,0,0,0,0,0,0,1


In [19]:
nan_count = all_mines_encoded.isna().sum()

nan_count_sorted = nan_count.sort_values(ascending=False)

print(nan_count_sorted.head(12))

reported_area              38662
permit_application_type    38658
edit_month                 21620
permit_approval_month      17741
highwall                   17028
inspectable_unit_status    16524
steep_slope                13740
mountaintop                13740
contour                    13740
area_mine                  13740
auger                      13740
post_smcra                 11812
dtype: int64


In [20]:
##### TESTING DROPSET
columns = all_mines_encoded.columns
column_to_exclude = ['trees', 'water', 'snow_and_ice','bare', 'built', 'shrub_and_scrub', 'crops']
columns = list(set(columns) - set(column_to_exclude))

selected_columns = all_mines_encoded[columns]

features = all_mines_encoded[columns]
target = all_mines_encoded['trees']

X, y = [], []


sequence_length = 10

for i in range(len(features) - sequence_length):
    X.append(features.iloc[i:i + sequence_length].values)
    y.append(target.iloc[i + sequence_length])

X = np.array(X)
y = np.array(y)

In [21]:
for i in all_mines_encoded.columns:
    if all_mines_encoded[i].isna().sum() > 0:
        print('{} has {}% NAs'.format(i,(all_mines_encoded[i].isna().sum() / len(all_mines_encoded))*100))

coalmine_op_status has 30.531271015467382% NAs
inspectable_unit_status has 42.739640991154104% NAs
post_smcra has 30.551963167968548% NAs
reported_area has 100.0% NAs
permit_application_type has 99.98965392374942% NAs
area_mine has 35.538771920749056% NAs
contour has 35.538771920749056% NAs
mountaintop has 35.538771920749056% NAs
steep_slope has 35.538771920749056% NAs
highwall has 44.04324659872743% NAs
auger has 35.538771920749056% NAs
contact has 30.531271015467382% NAs
Number_company_to_permit_id has 30.531271015467382% NAs
edit_month has 55.92054213439553% NAs
permit_approval_month has 45.88743469039367% NAs
WaterQuality has 14.000827686100045% NAs
Max Temperature has 13.993068128912109% NAs
Precipitation has 13.993068128912109% NAs


In [22]:
#very quick imputation
all_mines_encoded = all_mines_encoded.fillna(0)

In [149]:
#gut check using random forest

In [150]:
list(features.columns)

['Company_APPALACHIA MINING COMPANY',
 'Company_H & S COAL CO',
 'Company_PANTHER LAND CORPORATION',
 'Company_ZY COAL COMPANY',
 'Company_R C COAL COMPANY',
 'Company_BLUE GEM COAL & LAND CO',
 'Company_ST CLAIR HANOVER COAL CO',
 'Company_W & R COAL CO',
 'Company_SHAWN COAL COMPANY INC',
 'Company_DUSTY DIAMOND COMPANY INC',
 'Company_ENERGY INC',
 'Company_WILLCO ENTERPRIZES INC',
 'Company_WILLIAM & WILLARD COAL CO',
 'Company_MITCO ENTERPRISES, INC',
 'Company_STONE KING COAL CO INC',
 'Company_C & E COAL COMPANY',
 'Company_P & S MINING & CONST CO INC',
 'Company_NEWEAGLE MINING CORP.',
 'Company_H & H COAL SALES',
 'Company_EASTERN COAL CORPORATION',
 'Company_LITTLE T R COAL CO',
 'Company_COSTAIN COAL INC (II)',
 'Company_GREEN CONSTRUCTION OF INDIANA INC',
 'Company_LEXINGTON COAL COMPANY, LLC',
 'Company_SALYER COAL COMPANY',
 'Company_MERYL THORNSBURY',
 'Company_JINKS MINING, INC.',
 'Company_R & T COAL CO INC',
 'Company_BLACK MAGIC MINING INC',
 'Company_DIVORCE COAL CO

In [151]:
import pickle
from google.colab import files

# save the model
filename = 'rf_trees.pkl'
pickle.dump(rf, open(filename, 'wb'))
files.download('rf_trees.pkl')

NameError: ignored

In [1]:
##### TESTING DROPSET

columns = all_mines_encoded.columns
column_to_exclude = ['mine_name','permit_id','trees', 'water', 'snow_and_ice','bare', 'built', 'shrub_and_scrub', 'crops']
columns = list(set(columns) - set(column_to_exclude))

selected_columns = all_mines_encoded[columns]

features = all_mines_encoded[columns]
target = all_mines_encoded['trees']

X, y = [], []


sequence_length = 4

for i in range(len(features) - sequence_length):
    X.append(features.iloc[i:i + sequence_length].values)
    y.append(target.iloc[i + sequence_length])

X = np.array(X)
y = np.array(y)

NameError: ignored

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=0)



In [ ]:
feature_length = X_train.shape[2]

In [ ]:
feature_length

In [ ]:
sequence_length

In [ ]:
data

In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
import numpy as np

# define model
inputs1 = Input(shape=(2, 3))
lstm1, state_h, state_c = LSTM(1, return_sequences=True, return_state=True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])

# define input data
data = np.random.rand(2, 3)
data = data.reshape((1,2,3))

# make and show prediction
print(model.predict(data))

In [2]:
regressor = Sequential()
regressor.add(tf.keras.layers.Embedding(feature_length, 4, input_length=sequence_length))
regressor.add(LSTM(units = 32,
                    return_sequences = True,
                    activation = 'relu'))

regressor.add(LSTM(units = 32))
regressor.add(Dense(units = 1))
regressor.add(Dropout(0.10))

regressor.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics = ['mean_squared_error'])
regressor.fit(X_train, y_train, epochs = 25, batch_size = 32)

NameError: ignored

In [ ]:
regressor = tf.keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
regressor.add(tf.keras.layers.Embedding(input_dim=feature_length, output_dim=4))
regressor.add(LSTM(units = 32,
                    return_sequences = True,
                    activation = 'relu'))
regressor.add(Dense(units = 1))
regressor.add(Dropout(0.10))

regressor.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics = ['mean_squared_error'])
regressor.fit(X_train, y_train, epochs = 25, batch_size = 32)

In [ ]:
regressor = Sequential()
regressor.add(LSTM(units = 32, input_shape=(3, X_train.shape[2]), return_sequences = True, activation = 'relu'))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics = ['mean_squared_error'])
regressor.fit(X_train, y_train, epochs = 25, batch_size = 32)

In [ ]:
regEmb = tf.keras.Sequential()
regEmb.add(tf.keras.layers.Embedding(input_dim=X_train.shape[2],output_dim=1,input_length=4))
regEmb.add(LSTM(units = 32, input_shape=(4, X_train.shape[2]), return_sequences = True,  activation = 'relu'))
regEmb.add(Dropout(0.2))
regEmb.add(Dense(units = 1))
regEmb.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics = ['mean_squared_error'])
regEmb.fit(X_train, y_train, epochs = 25, batch_size = 32)


In [ ]:
y_pred = regressor.predict(X_test)



In [ ]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('\nMean Squared Error (MSE):', mse)
print('Mean Absolute Error (MAE):', mae)
print('R-squared (R^2):', r2)

In [ ]:
y_test

In [ ]:
res = (y_test - y_pred.tolist())
res = res.tolist()

In [ ]:
fig, axs = plt.subplots(ncols = 2,figsize = (8,4))
axs[0].scatter(x = y_pred, y = y_test,alpha = .5)
axs[0].set_title('Actual vs. Predicted Values')
axs[1].scatter(x = y_pred, y = res,alpha=.5)
axs[1].set_title('Residuals vs. Predicted Values')
fig.suptitle('Cross-Validated Predictions Plot')

In [ ]:
plt.plot(y_test,
         color = 'red',
         label = 'Acrual values')

plt.plot(y_pred,
         color = 'blue',
         label = 'Predicted values')

plt.legend()
plt.xlim(0, 50) # change to see everything
plt.title('Actual vs. Predicted Values')
plt.xlabel('Observation')
plt.ylabel('Value of Trees Variable')

plt.show()

In [ ]:
plt.scatter(range(len(y_test)),
            y_test,
            color = 'red',
            label = 'Actual Values')

plt.scatter(range(len(y_pred)),
            y_pred,
            color = 'blue',
            label = 'Predicted Values')

plt.legend()
plt.title('Actual vs. Predicted Values')
plt.xlim(0, 50) # change to see everything
plt.xlabel('Observation')
plt.ylabel('Value of Trees Variable')

plt.show()


In [ ]:
error = y_test - y_pred.flatten()
plt.hist(error, bins = 30)
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Errors')

plt.show()

# stack overflow
# A symmetric and narrow distribution around zero
# indicates better prediction accuracy.

In [ ]:
plt.scatter(y_test, error, color = 'red')
plt.axhline(y = 0,
            color = 'blue',
            linestyle = '--',
            linewidth = 2)

plt.xlabel('Actual Values')
plt.ylabel('Prediction Error')
plt.title('Prediction Error vs. Actual Values')

plt.show()


In [ ]:
# do we need to scale the data??

In [ ]:
# prompt: create random forest regression model on X_train, y_train, X_test, y_test

columns = all_mines_encoded.columns
column_to_exclude = ['mine_name','permit_id','trees', 'water', 'snow_and_ice','bare', 'built', 'shrub_and_scrub', 'crops']
columns = list(set(columns) - set(column_to_exclude))

selected_columns = all_mines_encoded[columns]

features = all_mines_encoded[columns]
target = all_mines_encoded['trees']

X = features
y = target

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=0)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('\nMean Squared Error (MSE):', mse)
print('Mean Absolute Error (MAE):', mae)
print('R-squared (R^2):', r2)